In [1]:
import glob
import numpy  as np
import tables as tb

import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# general IC imports
from invisible_cities.database          import load_db
from invisible_cities.core.system_of_units_c import units
adc, pes, mus = units.adc, units.pes, units.mus
NN = -999999

# IRENE
from invisible_cities.cities.components import deconv_pmt
from invisible_cities.cities.components import calibrate_pmts
from invisible_cities.cities.components import calibrate_sipms

from invisible_cities.cities.components import deconv_pmt
from invisible_cities.cities.components import calibrate_pmts
from invisible_cities.cities.components import calibrate_sipms
from invisible_cities.cities.components import zero_suppress_wfs

from invisible_cities.reco.peak_functions import split_in_peaks
from invisible_cities.reco.peak_functions import select_peaks
from invisible_cities.reco.peak_functions import select_wfs_above_time_integrated_thr
from invisible_cities.reco.peak_functions import pick_slice_and_rebin

from invisible_cities.types.ic_types import minmax

# PENTHESILEA
from invisible_cities.reco.peak_functions import rebin_times_and_waveforms

# ESMERALDA
from invisible_cities.reco.corrections_new import read_maps
from invisible_cities.reco.corrections_new import apply_all_correction
from invisible_cities.reco.corrections_new import norm_strategy

## General cut parameters

In [2]:
# S1 selection
s1emin = 40
s1wmin = 175

pmt_ids = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

## Import raw waveforms

In [3]:
run = 7430
wfs_dir        = f"/home/gdiaz/Verify_IRENENB_corrections/DATA/{run}/rwf/"
wfs_files = glob.glob( wfs_dir + "/*" )
wfs_files.sort()

# IRENE

## Configuration Parameters

In [4]:
files_in  = wfs_files
file_out  = ""

run = 7430
n_baseline = 62400 

n_mau   = 100
thr_mau =   3 * adc

thr_csum_s1 = 0.5 * pes
thr_csum_s2 = 2.0 * pes

thr_sipm   = 1.0 * pes    
thr_sipm_type = "Common"

s1_tmin   =   0 * mus 
s1_tmax   = 790 * mus 
s1_stride =   4       
s1_lmin   =   5       
s1_lmax   =  30      
s1_rebin_stride = 1

s2_tmin   =      0 * mus 
s2_tmax   =   1601 * mus 
s2_stride =     40      
s2_lmin   =     80       
s2_lmax   = 200000       
s2_rebin_stride = 40   

thr_sipm_s2 = 5 * pes  

detector_db = "new" ## Added by me

if thr_sipm_type.lower() == "common": 
    sipm_thr = thr_sipm

### Select file and waveform

In [9]:
file_number = 0

RWFs_file = tb.open_file(files_in[file_number])
pmt_rwfs_all  = RWFs_file.root.RD.pmtrwf
sipm_rwfs_all = RWFs_file.root.RD.sipmrwf
time_stamps   = RWFs_file.root.Run.events

#select random waveform
i = int( np.random.random()* (len(time_stamps) - 1) )
event_time = time_stamps[i]


pmt_rwfs  = pmt_rwfs_all [i]
sipm_rwfs = sipm_rwfs_all[i]

RWFs_file.close()

### Sensors processing

In [10]:
rwf_to_cwf = deconv_pmt    (detector_db, run, n_baseline)
pmt_cwfs   = rwf_to_cwf    (pmt_rwfs)
cwf_to_ccwf = calibrate_pmts(detector_db, run, n_mau, thr_mau)
pmt_ccwfs, ccwfs_mau, cwf_sum, cwf_sum_mau  = cwf_to_ccwf    (pmt_cwfs)

sipm_rwf_to_cal = calibrate_sipms(detector_db, run, sipm_thr)
sipm_cwfs = sipm_rwf_to_cal(sipm_rwfs)

### Find S1 and S2 peaks (IRENE)

In [11]:
zero_suppress = zero_suppress_wfs(thr_csum_s1, thr_csum_s2)
s1_indices, s2_indices = zero_suppress(cwf_sum, cwf_sum_mau)

indices_split   = split_in_peaks(s1_indices, s1_stride)
time    = minmax(min = s1_tmin, max = s1_tmax)
length  = minmax(min = s1_lmin, max = s1_lmax)
s1_selected_splits = select_peaks  (indices_split, time, length)
print("Number of S1 candidates:", len(s1_selected_splits))


indices_split   = split_in_peaks(s2_indices, s2_stride)
time    = minmax(min = s2_tmin, max = s2_tmax)
length  = minmax(min = s2_lmin, max = s2_lmax)
s2_selected_splits = select_peaks  (indices_split, time, length)
print("Number of S2 candidates:", len(s2_selected_splits))

Number of S1 candidates: 1
Number of S2 candidates: 1


### S1 and S2 cuts

In [12]:
# S1 cuts
if len(s1_selected_splits)==0:
    raise Exception("No S1 in event")
    
s1es, s1ws = [], []
for ss in s1_selected_splits:
    s1_pmt = np.sum( pmt_ccwfs[:, ss[0]: ss[-1]], axis=0)
    s1es.append( np.sum(s1_pmt)    )
    s1ws.append( (ss[-1]-ss[0])*25 )
s1es, s1ws = np.array(s1es), np.array(s1ws)

sel = (s1es>=s1emin) & (s1ws>=s1wmin)
idxs = np.argwhere(sel).flatten()

if len(idxs)==0:
    raise Exception("No S1 pass the cut")
elif len(idxs)>1:
    raise Exception("More than 1S1 pass the cut")
else:
    idx = idxs[0]
    s1_pmt = np.sum( pmt_ccwfs[:, s1_selected_splits[idx][0]: s1_selected_splits[idx][-1]], axis=0)
    times  = np.arange(s1_selected_splits[idx][0], s1_selected_splits[idx][-1])*25

    S1_time = times[np.argmax(s1_pmt)]
    
# S2 cut
if len(s2_selected_splits)>1:
    raise Exception("More that 1S2")

### Select and process S2 RWFs

In [13]:
times  = np.arange     (pmt_ccwfs.shape[1]) * 25 * units.ns
widths = np.full       (pmt_ccwfs.shape[1],   25 * units.ns)

indices = s2_selected_splits[0]
wfs     = pmt_ccwfs

with_sipms = True

times, rebinned_widths, s2_pmts = pick_slice_and_rebin(indices, times, widths,
                                                       wfs, 
                                                       rebin_stride = s2_rebin_stride, 
                                                       pad_zeros    = with_sipms)

In [14]:
s2_sipms = sipm_cwfs[:, s2_selected_splits[0][0] //40 : s2_selected_splits[0][-1]//40 + 1]
# thr_sipm_s2
sipm_ids, s2_sipms = select_wfs_above_time_integrated_thr(s2_sipms, thr_sipm_s2)



#float32
s2_pmts  = np.float32( s2_pmts )
s2_sipms = np.float32( s2_sipms)
times    = np.float32( times   )

#pmt ids
c = np.zeros(s2_pmts.shape[0])
c[pmt_ids] = 1
s2_pmts  = np.multiply( c, s2_pmts.T ).T

## PENTHESILEA

## Rebin

In [15]:
# Rebin
rebin = 2

_, _, s2_sipms    = rebin_times_and_waveforms(times, rebinned_widths, s2_sipms,
                                              rebin_stride=rebin, slices=None)

times, _, s2_pmts = rebin_times_and_waveforms(times, rebinned_widths, s2_pmts,
                                              rebin_stride=rebin, slices=None)

In [16]:
s2_pmts_penth  = np.copy( s2_pmts )
s2_sipms_penth = np.copy( s2_sipms )

## Create hits

In [17]:
datasipm = load_db.DataSiPM("new", run)
sipm_xs  = datasipm.X.values
sipm_ys  = datasipm.Y.values
n_sipms = len(sipm_ids)

X, Y = sipm_xs[sipm_ids], sipm_ys[sipm_ids]
T = (times - S1_time)/1000

In [18]:
E_per_slice = np.sum( s2_pmts_penth, axis=0)

hits = []
for t, e, q in zip(T, E_per_slice, s2_sipms_penth.T):
    hits.append( (X, Y, np.full( n_sipms, t), 
                  np.full(n_sipms, e), np.full( n_sipms, -1), 
                  q                  , np.full( n_sipms, -1) ) )

hits = np.array( hits )
hits = np.swapaxes(hits, axis1=1, axis2=2)
hits = np.concatenate( hits )

In [19]:
H = np.array(np.zeros(np.shape(hits)[0]), 
             dtype=[("X", int)  , ("Y", int)  , ("Z", float), 
                    ("E", float), ("Ec",float),
                    ("Q", float), ("Qc", float)])

H["X"], H["Y"], H["Z"] = hits[:, 0], hits[:, 1], hits[:, 2]
H["E"], H["Ec"] = hits[:, 3], -1                            #OJO, la energía del hit es la energia de la slice
H["Q"], H["Qc"] = hits[:, 5], -1

#remove 0 charge hits
sel = ~(H["Q"]==0)
hits = H[sel]

## Charge Corrections

In [20]:
from invisible_cities.reco.corrections_new import correct_lifetime_
from invisible_cities.reco.corrections_new import maps_coefficient_getter

qmap_file = glob.glob( f"/home/gdiaz/ANALYSIS/data/maps/qmap_*")[0]
qmaps = read_maps( qmap_file )

# total_correction = apply_all_correction(maps, apply_temp=True,
#                                         norm_strat=norm_strategy.kr)

get_lt_corr_fun = maps_coefficient_getter(qmaps.mapinfo, qmaps.lt)
apply_temp = False
if apply_temp:
    raise Exception("Apply temp is False")
else:
    ltevol_vs_t = lambda x : np.ones_like(x)

In [21]:
X, Y, Z = hits["X"], hits["Y"], hits["Z"]
E = hits["E"]
Q = hits["Q"]

T = np.full(len(hits), event_time[-1]/1000)

lt_factor  = correct_lifetime_(Z, get_lt_corr_fun(X, Y) * ltevol_vs_t(T))
Qc = lt_factor * Q

hits["Qc"] = Qc

## Charge cut

In [22]:
qth_penth = 5 * pes

In [23]:
qth = qth_penth

sel = (hits["Qc"]>=qth)
hits["Qc"][~sel] = 0

slides = np.unique( hits["Z"] )
for slide in slides:
    sel = (hits["Z"]==slide)
    slide_hits = hits[sel]
    
    #q = slide_hits["Qc"]
    e = slide_hits["E"]
    slide_e = e[0]     ## OJO AQUÍ   

    if np.sum( q ) == 0:
        idxs = np.argwhere(sel).flatten()
        hits = np.delete(hits, idxs)
        hits = np.insert(hits, 0, (0, 0, slide, 
                                   slide_e, NN, 
                                   NN, NN))
#     else:
#         hits["E"][sel] = slide_e * q / np.sum(q)
        
sel = (hits["Qc"]==0)
hits = np.delete( hits, np.argwhere(sel))
hits = np.sort(hits, order="Z")

In [26]:
hits

array([(-175,   95, 260.56503138, 1.02639031e+03, -1.00000e+00,  5.75496960e+00,  6.18677109e+00),
       (-175,  105, 260.56503138, 1.02639031e+03, -1.00000e+00,  4.85948896e+00,  5.22410159e+00),
       (-165,   95, 260.56503138, 1.02639031e+03, -1.00000e+00,  7.15794849e+00,  7.69501699e+00),
       (-165,  105, 260.56503138, 1.02639031e+03, -1.00000e+00,  6.00877190e+00,  6.65412423e+00),
       (-155,   95, 260.56503138, 1.02639031e+03, -1.00000e+00,  4.91758823e+00,  5.28656011e+00),
       (-155,  105, 260.56503138, 1.02639031e+03, -1.00000e+00,  8.35722542e+00,  8.98427694e+00),
       (-185,   85, 262.45173337, 6.46438474e+03, -1.00000e+00,  8.13998604e+00,  8.75532330e+00),
       (-185,  105, 262.45173337, 6.46438474e+03, -1.00000e+00,  8.59989262e+00,  9.24999624e+00),
       (-175,   85, 262.45173337, 6.46438474e+03, -1.00000e+00,  9.09796906e+00,  9.78572446e+00),
       (-175,   95, 262.45173337, 6.46438474e+03, -1.00000e+00,  1.37571182e+01,  1.47970792e+01),
       (-1

In [27]:
# JOIN NN hits

sel = (hits["Q"]==NN)

nn_hits = hits[ sel]
hits    = hits[~sel]

slides = np.unique( hits["Z"] )
for nn_hit in nn_hits:
    
    #select slide to append
    d = np.abs( slides - nn_hit["Z"] ) 
    
    slide = slides[ np.argmin( d ) ]
    slide_hits = hits[hits["Z"]==slide]
    
    #new energy
    hits["E"][hits["Z"] == slide] = slide_hits["E"] + nn_hit["E"]

In [30]:
hits

array([(-175,   95, 260.56503138,  1026.39031315, -1.,   5.7549696 ,   6.18677109),
       (-175,  105, 260.56503138,  1026.39031315, -1.,   4.85948896,   5.22410159),
       (-165,   95, 260.56503138,  1026.39031315, -1.,   7.15794849,   7.69501699),
       (-165,  105, 260.56503138,  1026.39031315, -1.,   6.0087719 ,   6.65412423),
       (-155,   95, 260.56503138,  1026.39031315, -1.,   4.91758823,   5.28656011),
       (-155,  105, 260.56503138,  1026.39031315, -1.,   8.35722542,   8.98427694),
       (-185,   85, 262.45173337,  6464.38473511, -1.,   8.13998604,   8.7553233 ),
       (-185,  105, 262.45173337,  6464.38473511, -1.,   8.59989262,   9.24999624),
       (-175,   85, 262.45173337,  6464.38473511, -1.,   9.09796906,   9.78572446),
       (-175,   95, 262.45173337,  6464.38473511, -1.,  13.75711823,  14.79707916),
       (-175,  105, 262.45173337,  6464.38473511, -1.,  21.59159279,  23.22379603),
       (-175,  115, 262.45173337,  6464.38473511, -1.,   7.33725786,   7.891